In [1]:
# ==============================================================================
# CrossFit 코칭 데모 (Colab / UI 조정판)
# ------------------------------------------------------------------------------
# 이 스크립트는 Gradio 프레임워크를 사용하여 크로스핏 코칭 데모 UI를 구축하고,
# Colab 환경에서 실행 가능하도록 포트 탐색 및 공유 기능을 포함합니다.
# 주요 기능: 챗봇(Q&A), 영상 코칭(데모), 개인 맞춤 추천, 용어/규칙 검색,
# 식단/회복 가이드, 인증/챌린지 안내, 멘토링, 근거 자료 허브.
# ==============================================================================

In [2]:
# 1) 필요한 패키지 설치
# 충돌을 최소화하고 특정 버전 유지를 위해 Gradio만 명시하여 설치합니다.
# Colab 환경에서만 실행되며, 이미 설치된 경우 스킵됩니다.
!pip -q install "gradio==4.44.1"
!pip -q install "pydantic==2.10.6" # pydantic 의존성 문제 해결을 위해 추가

!pip install langchain langchain_community langsmith chromadb openai tiktoken pypdf
!pip install -U langchain-openai

## DB 세팅

In [3]:
# Google Drive 연동
from google.colab import drive, userdata
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sqlite3
import os
import hashlib

db_dir = "/content/drive/MyDrive/2조_3rd_프로젝트/sqlite_db"
os.makedirs(db_dir, exist_ok=True)
db_path = os.path.join(db_dir, "users.db")

def init_userdb():
    with sqlite3.connect(db_path) as conn:
        cur = conn.cursor()

        # 유저 테이블
        cur.execute("""
        CREATE TABLE IF NOT EXISTS users (
          email TEXT PRIMARY KEY,
          nickname TEXT,
          password_hash TEXT,
          created_at TEXT,
          role TEXT DEFAULT "user"
        );
        """)

        # 로그인/로그아웃 로그
        cur.execute("""
        CREATE TABLE IF NOT EXISTS session_logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            email TEXT,
            action TEXT,
            ts TEXT
        )
        """)

        # QA 로그
        cur.execute("""
        CREATE TABLE IF NOT EXISTS qa_logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            email TEXT,
            question TEXT,
            answer TEXT,
            ts TEXT
        )
        """)

        admin_email, admin_nickname, admin_pw = "admin@crossfit.com", "관리자", "admin123"
        pw_hash = hashlib.sha256(admin_pw.encode()).hexdigest()
        cur.execute("INSERT OR IGNORE INTO users(email, nickname, password_hash, created_at, role) VALUES (?, ?, ?, datetime('now'), 'admin')", (admin_email, admin_nickname, pw_hash))
        demo_email, demo_nickname, demo_pw = "demo@demo.com", "데모", "x"
        pw_hash = hashlib.sha256(demo_pw.encode()).hexdigest()
        cur.execute("INSERT OR IGNORE INTO users(email, nickname, password_hash, created_at) VALUES (?, ?, ?, datetime('now'))", (demo_email, demo_nickname, pw_hash))
        conn.commit()

init_userdb()

# 유저 login, logout, signup 히스토리
def log_session(email, action):
    with sqlite3.connect(db_path) as conn:
        cur = conn.cursor()
        cur.execute("INSERT INTO session_logs (email, action, ts) VALUES (?, ?, ?)",
                    (email, action, datetime.now().isoformat()))
        conn.commit()

# 유저 QA 히스토리
def log_qa(email, question, answer):
    with sqlite3.connect(db_path) as conn:
        cur = conn.cursor()
        cur.execute(
            "INSERT INTO qa_logs (email, question, answer, ts) VALUES (?, ?, ?, ?)",
            (email, question, answer, datetime.now().isoformat())
        )
        conn.commit()


In [5]:
# conn = sqlite3.connect(db_path)
# cur = conn.cursor()

# cur.execute("SELECT * FROM session_logs;")
# print(cur.fetchall())
# conn.close()

In [6]:
def get_user(email):
    conn = sqlite3.connect(db_path); c = conn.cursor()
    c.execute("SELECT email, nickname, password_hash, role FROM users WHERE email=?", (email,))
    user = c.fetchone(); conn.close()
    return user

def register_user(email, nickname, pw_hash):
    with sqlite3.connect(db_path) as conn:
        c = conn.cursor()
        c.execute("INSERT OR IGNORE INTO users(email, nickname, password_hash, created_at) VALUES (?, ?, ?, datetime('now'))", (email, nickname, pw_hash))
        conn.commit()

def login_user(email, pw):
    user = get_user(email)
    if not user: return None, "존재하지 않는 계정"
    import hashlib
    if hashlib.sha256(pw.encode()).hexdigest() != user[2]:
        return None, "비밀번호 오류"
    session = {"user_id": user[0], "name": user[1], "role": user[3]}

    return session, f"{user[1]}님, 환영합니다."

def is_admin(session):
    return session and session.get("role") == "admin"


In [7]:
import os

# 가이드 PDF 폴더, DB 폴더 경로
watch_dir = "/content/drive/MyDrive/2조_3rd_프로젝트/크로스핏_가이드"
chroma_dir = "/content/drive/MyDrive/2조_3rd_프로젝트/chroma_db"
chroma_backup_dir = "/content/drive/MyDrive/2조_3rd_프로젝트/chroma_db_backups"
os.makedirs(chroma_dir, exist_ok=True)
os.makedirs(chroma_backup_dir, exist_ok=True)

# OpenAI API 클라이언트 생성
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "ai_camp_3rd_project"

# 벡터DB가 이미 존재하면 skip
def chroma_db_exists(chroma_dir):
    return os.path.isfile(os.path.join(chroma_dir, "chroma.sqlite3"))

if not chroma_db_exists(chroma_dir):
    from langchain.document_loaders import PyPDFLoader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain_openai import OpenAIEmbeddings
    from langchain.vectorstores import Chroma
    import tiktoken

    embeddings = OpenAIEmbeddings()

    # --- 토큰 수 기반 batch 함수 ---
    def batch_by_token_limit(chunks, max_tokens=290000, model_name="text-embedding-3-small"):
        tokenizer = tiktoken.encoding_for_model(model_name)
        curr_batch = []
        curr_tokens = 0
        for chunk in chunks:
            text = chunk.page_content if hasattr(chunk, "page_content") else str(chunk)
            tokens = len(tokenizer.encode(text))
            # batch를 넘으면 새 batch 시작
            if curr_tokens + tokens > max_tokens and curr_batch:
                yield curr_batch
                curr_batch = []
                curr_tokens = 0
            curr_batch.append(chunk)
            curr_tokens += tokens
        if curr_batch:
            yield curr_batch

    file_paths = [
        os.path.join(watch_dir, f)
        for f in os.listdir(watch_dir)
        if f.lower().endswith('.pdf')
    ]  # 크로스핏 가이드 PDF 파일들
    docs = []

    for file in file_paths:
        # PDF 자동 추출
        loader = PyPDFLoader(file)
        docs.extend(loader.load())

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    all_chunks = splitter.split_documents(docs)

    for i, batch_chunks in enumerate(batch_by_token_limit(all_chunks, 290000)):
        db = Chroma.from_documents(
            documents=batch_chunks,
            embedding=embeddings,
            persist_directory=chroma_dir
        )
        db.persist()
        del db
        print(f"Batch {i+1} 저장 완료 (chunks: {len(batch_chunks)})")
    print("✅ 신규 임베딩 및 VectorDB 생성 완료")
else:
    print("⚡ 기존 VectorDB가 이미 존재합니다. batch/bulk 임베딩 건너뜀.")

⚡ 기존 VectorDB가 이미 존재합니다. batch/bulk 임베딩 건너뜀.


In [8]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()

# VectorDB 호출
vectordb = Chroma(
    persist_directory=chroma_dir,
    embedding_function=embeddings
)

print("VectorDB 로딩 완료!")

/tmp/ipython-input-3957547275.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


VectorDB 로딩 완료!


In [9]:
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    model_name="gpt-4.1-mini",
    temperature=0.3
)

# 프롬프트 템플릿 객체
prompt = ChatPromptTemplate.from_template(
"""
아래 문서 참고(context)해서 반드시 한국어로만, 구체적으로 답해줘.
<context>
{context}
</context>
질문: {question}
"""
)

retriever = vectordb.as_retriever(search_kwargs={"k": 4})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": prompt
    }
)

# # 질의 예시
# query = "크로스핏 입문자가 반드시 알아야 할 용어를 알려줘."
# result = qa_chain({"query": query})
# print("AI 답변:", result["result"])
# print("\n참고 chunk:", result["source_documents"])

/tmp/ipython-input-960068463.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


## Gradio UI

### 기본 함수

In [10]:
# 필요한 파이썬 내장/외부 라이브러리 임포트
import gradio as gr          # Gradio: 웹 기반 UI를 쉽게 만들 수 있는 라이브러리
import json                  # JSON 데이터 직렬화/역직렬화
import time                  # 시간 관련 기능 (예: sleep)
import uuid                  # 고유 식별자(UUID) 생성 (예: 세션 키, 파일명)
import random                # 무작위 기능 (예: 포트 번호 셔플)
import socket                # 네트워크 소켓 프로그래밍 (예: 사용 가능한 포트 찾기)
from datetime import datetime # 날짜 및 시간 객체 다루기
import shutil

# ------------------------------------------------------------------------------
# 포트 탐색 유틸리티
# Colab 환경에서 Gradio 앱 실행 시 포트 충돌을 회피하기 위한 기능입니다.
# ------------------------------------------------------------------------------
def find_free_port(start: int = 7860, end: int = 7960) -> int | None:
    """
    지정된 범위 내에서 사용 가능한 네트워크 포트를 찾아 반환합니다.
    Colab 환경에서 여러 Gradio 앱 실행 시 포트 충돌을 방지하기 위해 사용됩니다.

    Args:
        start (int): 탐색을 시작할 포트 번호. 기본값 7860.
        end (int): 탐색을 종료할 포트 번호. 기본값 7960.

    Returns:
        int | None: 사용 가능한 포트 번호 (정수) 또는 찾지 못한 경우 None.
    """
    ports = list(range(start, end)) # 시작부터 끝까지의 포트 번호 리스트 생성
    random.shuffle(ports)           # 포트 탐색 순서를 무작위로 섞어 충돌 가능성 줄임

    for p in ports:
        # IPv4, TCP 소켓 생성
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            # 소켓이 닫힌 후 동일 주소(IP:Port)를 즉시 재사용할 수 있도록 설정
            # Colab 환경에서 커널 재시작 없이 앱 재실행 시 유용합니다.
            s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            try:
                # 0.0.0.0 주소에 포트를 바인딩하여 모든 네트워크 인터페이스에서 접근 허용
                s.bind(("0.0.0.0", p))
                return p # 바인딩에 성공하면 해당 포트 반환
            except OSError:
                # 포트가 이미 사용 중인 경우, 다음 포트로 시도
                continue
    return None # 사용 가능한 포트를 찾지 못한 경우

# ------------------------------------------------------------------------------
# 전역 상태 (Global State) / 초기 데이터 정의
# ------------------------------------------------------------------------------

# 'app'은 전체 데모 앱의 상태를 저장하는 전역 딕셔너리입니다.
# 실제 웹 서비스에서는 각 사용자 세션별로 분리된 상태 관리가 필요하지만,
# 이 데모에서는 단일 사용자 환경을 가정하여 전역 변수로 관리합니다.

def seed_sources() -> list[dict]:
    """
    데모 앱에서 초기 근거 자료(소스) 목록을 정의합니다.
    실제 링크는 과제 환경에 맞게 교체 가능하며, 여기서는 예시 링크를 사용합니다.

    Returns:
        list[dict]: 각 딕셔너리는 'title'과 'url' 키를 가집니다.
    """
    return [
        {"title": "CrossFit Level 1 Training Guide (PDF)", "url": "https://example.com/cf-level1.pdf"},
        {"title": "CrossFit Journal - Movement Standards", "url": "https://journal.crossfit.com/example-standards"},
        {"title": "USAW Technique Guide", "url": "https://usaweightlifting.org/example-technique"}
    ]

def seed_glossary() -> list[dict]:
    """
    크로스핏 관련 용어집 초기 데이터를 정의합니다.

    Returns:
        list[dict]: 각 딕셔너리는 'term', 'category', 'desc' 키를 가집니다.
    """
    return [
        {"term": "박스", "category": "용어", "desc": "크로스핏 체육관을 의미"},
        {"term": "WOD", "category": "용어", "desc": "Workout of the Day, 하루 운동 프로그램"},
        {"term": "온램프", "category": "프로그램", "desc": "초보자 적응 과정"}
    ]

def init_states() -> dict:
    """
    데모 앱의 모든 전역 상태 변수를 초기화합니다.
    로그아웃 시 앱 상태를 초기 상태로 되돌리는 데 사용됩니다.

    Returns:
        dict: 초기화된 앱 상태 딕셔너리.
    """
    return {
        "user_session": {
            "user_id": None,     # 현재 로그인한 사용자의 ID
            "name": None,        # 현재 로그인한 사용자의 닉네임
            "auth": False,       # 인증 여부 (로그인 상태)
            "session_key": None  # 세션 고유 키 (데모용, 실제 환경에서는 더 보안 강화 필요)
        },
        "chat_history": [],        # 챗봇과의 대화 기록을 저장하는 리스트
                                   # 각 요소는 {"role": "user"|"assistant", "content": "...", "sources": [...]} 형태
        "source_bucket": {},       # 중복 없이 근거 자료 URL과 타이틀을 저장하는 딕셔너리
                                   # key: url, value: {"title": str|None, "first_seen": iso_datetime}
        "video_history": [],       # 영상 코칭 분석 결과를 저장하는 리스트
        "recommend_history": [],   # 개인 맞춤 추천 기록을 저장하는 리스트
        "evidence_library": [],    # 모든 공식 자료 및 세션 중 적립된 링크를 모아둔 리스트
        "glossary": seed_glossary(),  # 크로스핏 용어집
        "preset_sources": seed_sources() # 초기 로드될 근거 자료 목록
    }

# 스크립트 로드 시 'app' 변수가 존재하지 않으면 초기화합니다.
# Gradio 앱의 Hot Reloading(새로고침) 기능 때문에 여러 번 초기화될 수 있는 것을 방지합니다.
try:
    app
except NameError:
    app = init_states()

# ------------------------------------------------------------------------------
# 공통 유틸리티 함수들
# ------------------------------------------------------------------------------
def now_iso() -> str:
    """
    현재 시간을 ISO 8601 형식 문자열 (초 단위)로 반환합니다.

    Returns:
        str: ISO 8601 형식의 현재 시각 문자열. (예: "2025-09-16T15:30:00")
    """
    return datetime.now().isoformat(timespec="seconds")

def add_sources(sources: list[dict] | None):
    """
    챗봇 응답에서 받은 근거 자료(링크)들을 전역 evidence_library에 중복 없이 추가합니다.
    source_bucket을 사용하여 중복 체크 및 메타데이터(first_seen)를 기록합니다.

    Args:
        sources (list[dict] | None): 추가할 근거 자료 딕셔너리 리스트.
                                      각 딕셔너리는 'title'과 'url' 키를 포함합니다.
    """
    if not sources:
        return # sources가 없으면 아무것도 하지 않음

    for s in sources:
        url = s.get("url")
        title = s.get("title")
        if not url:
            continue # URL이 없는 자료는 스킵

        # source_bucket에 URL이 없으면 새로 추가하고, 최초 발견 시간 기록
        if url not in app["source_bucket"]:
            app["source_bucket"][url] = {"title": title, "first_seen": now_iso()}

        # evidence_library에 해당 URL이 없으면 전체 목록에 추가
        if not any(item["url"] == url for item in app["evidence_library"]):
            app["evidence_library"].append({"title": title or url, "url": url})

def sources_md() -> str:
    """
    현재 적립된 모든 근거 자료들을 Markdown 형식의 리스트로 렌더링합니다.
    '근거 자료 허브' 탭과 챗봇 응답 아래 '링크 모아보기' 패널에 사용됩니다.

    Returns:
        str: Markdown 형식의 근거 자료 리스트 문자열.
             근거 자료가 없으면 "아직 근거 자료가 없습니다." 메시지 반환.
    """
    if not app["evidence_library"]:
        return "아직 근거 자료가 없습니다."
    return "\n".join(
        f"- [{item.get('title') or item['url']}]({item['url']})"
        for item in app["evidence_library"]
    )

def export_chat_json() -> str:
    """
    현재까지의 챗봇 대화 히스토리와 사용자 세션 정보를 JSON 문자열로 직렬화합니다.
    사용자가 대화 기록을 다운로드할 때 사용됩니다.

    Returns:
        str: JSON 형식으로 직렬화된 대화 기록 문자열. (한글 깨짐 방지 처리됨)
    """
    payload = {
        "exported_at": now_iso(),        # 내보내진 시간
        "user_session_info": app["user_session"], # 사용자 세션 정보
        "chat_turns": app["chat_history"] # 대화 턴(turn) 목록
    }
    # 한글이 깨지지 않도록 ensure_ascii=False 설정, 가독성을 위해 indent=2 설정
    return json.dumps(payload, ensure_ascii=False, indent=2)

def tmp_filename(prefix: str = "chat_history", ext: str = "json") -> str:
    """
    고유한 임시 파일 이름을 생성합니다.
    챗봇 히스토리 다운로드 파일명 등에 사용됩니다.

    Args:
        prefix (str): 파일 이름의 접두사.
        ext (str): 파일의 확장자.

    Returns:
        str: 고유한 파일 이름 문자열. (예: "chat_history_a1b2c3d4.json")
    """
    return f"{prefix}_{uuid.uuid4().hex}.{ext}"

# ------------------------------------------------------------------------------
# 인증 및 세션 관리
# ------------------------------------------------------------------------------
def hash_pw(pw):  # 단순 해시 예시
    import hashlib
    return hashlib.sha256(pw.encode("utf-8")).hexdigest()

def do_login(email: str, name: str, pwd: str) -> tuple:
    """
    사용자 로그인 처리를 수행합니다.
    로그인 성공 시 사용자 세션 정보를 업데이트하고, 사전 정의된 근거 자료를 로드합니다.

    Args:
        email (str): 사용자가 입력한 이메일.
        name (str): 사용자가 입력한 닉네임.
        pwd (str): 사용자가 입력한 비밀번호.

    Returns:
        tuple: (entry_page_visibility, main_page_visibility, welcome_message_update, admin_tab_visibility)
               Gradio UI 컴포넌트의 가시성 업데이트 및 환영 메시지.
    """

    if not email or not pwd:
        # 이메일 또는 비밀번호가 비어있을 경우 에러 메시지 반환
        return (
            gr.update(visible=True),
            gr.update(visible=False),
            gr.update(value="이메일/비밀번호를 입력해 주세요."),
            gr.update(visible=False),
        )

    session_val, msg = login_user(email, pwd)
    vis_admin = is_admin(session_val)
    user = get_user(email)

    if not user:
      return (
          gr.update(visible=True),
          gr.update(visible=False),
          gr.update(value="가입되어 있지 않습니다. 회원가입 후 시도해주세요."),
          gr.update(visible=False),
          )
    elif hash_pw(pwd) != user[2]:
      return (
          gr.update(visible=True),
          gr.update(visible=False),
          gr.update(value="비밀번호가 일치하지 않습니다."),
          gr.update(visible=False),
          )

    # 실제 환경에서는 데이터베이스 조회 및 비밀번호 해싱/검증 로직이 들어갑니다.
    # 데모용이므로 간단하게 세션 정보만 업데이트합니다.
    app["user_session"] = {
        "user_id": user[0],
        "name": user[1] or "데모", # 닉네임이 없으면 '데모'로 설정
        "auth": True,          # 인증 상태를 True로 설정
        "session_key": f"sk-{uuid.uuid4().hex}" # 고유 세션 키 생성
    }
    # 로그인 성공 시, 미리 정의된 근거 자료(preset_sources)를 evidence_library에 복사
    app["evidence_library"] = list(app["preset_sources"])

    log_session(email, "login")

    # 로그인 성공 시, 진입 페이지 숨기고 메인 페이지를 보여줍니다.
    # welcome 메시지를 gr.update() 객체로 명시적으로 반환합니다.
    return (
        gr.update(visible=False),
        gr.update(visible=True),
        gr.update(value=f"{msg}"),
        gr.update(visible=vis_admin)
    )

def do_signup(email: str, name: str, pwd: str) -> tuple:
    """
    사용자 회원가입 처리를 수행합니다.
    이 데모에서는 회원가입 시 별도의 가입 절차 없이 바로 로그인 처리합니다.

    Args:
        email (str): 사용자가 입력한 이메일.
        name (str): 사용자가 입력한 닉네임.
        pwd (str): 사용자가 입력한 비밀번호.

    Returns:
        tuple: do_login 함수의 반환 값과 동일.
    """
    register_user(email, name or email, hash_pw(pwd))

    log_session(email, "signup")
    # 시연용: 회원가입은 곧바로 로그인으로 간주
    return do_login(email, name, pwd)

def do_logout() -> tuple:
    """
    사용자 로그아웃 처리를 수행합니다.
    앱의 모든 상태를 초기화하고, 로그인 페이지로 되돌립니다.

    Returns:
        tuple: (entry_page_visibility, main_page_visibility, status_message)
               Gradio UI 컴포넌트의 가시성 업데이트 및 로그아웃 메시지.
    """
    global app # 전역 변수 'app'을 수정하기 위해 global 선언

    log_session(app["user_session"]["user_id"], "logout")

    app = init_states() # 앱의 모든 상태를 초기화
    # 로그아웃 후 진입 페이지를 보여주고 메인 페이지를 숨깁니다.

    return (
        gr.update(visible=True),
        gr.update(visible=False),
        "로그아웃 되었습니다. 다시 로그인해 주세요.",
        gr.update(value=""),       # email 텍스트박스 공란화
        gr.update(value=""),       # name
        gr.update(value="")        # pwd
      )

# ------------------------------------------------------------------------------
# 챗봇 (Q&A) 기능
# ------------------------------------------------------------------------------
def safe_str(text):
    if text is None:
        return ""
    if not isinstance(text, str):
        return str(text)
    return text

def send_chat(user_msg: str) -> tuple:
    """
    사용자의 질문을 받아 챗봇 응답을 생성하고 대화 기록을 업데이트합니다.
    (현재는 모킹된 응답을 사용하며, 실제 환경에서는 LLM 백엔드를 호출합니다.)

    Args:
        user_msg (str): 사용자가 입력한 질문 메시지.

    Returns:
        tuple: (chat_history, status_message, links_panel_update, evidence_full_update)
               Gradio UI 컴포넌트 업데이트에 필요한 값들.
    """
    # 1. 로그인 상태 확인
    if not app["user_session"]["auth"]:
        return app["chat_history"], "로그인 후 이용해 주세요.", gr.update(value=sources_md()), gr.update(value=sources_md())
    # 2. 질문 입력 여부 확인
    if not user_msg or not user_msg.strip():
        return app["chat_history"], "질문을 입력해 주세요.", gr.update(value=sources_md()), gr.update(value=sources_md())

    # # 사용자 메시지를 대화 기록에 추가
    # app["chat_history"].append({"role": "user", "content": user_msg})

    # # --------------------------------------------------------------------------
    # # 실제 환경에서는 이 부분에서 GPT-4.1-mini + RAG(검색 증강 생성) 백엔드를 호출하여
    # # 질문에 대한 답변과 관련 근거 자료를 받아오게 됩니다.
    # # 현재는 데모를 위해 고정된 모킹(mocking) 응답을 사용합니다.
    # # --------------------------------------------------------------------------
    # time.sleep(0.1) # 챗봇 응답이 오는 것 같은 약간의 지연 효과
    # resp = {
    #     "answer_text": "공식 가이드 기준: 초기 당길 때 무릎의 과도한 전방 이동을 피하고, 힙-무릎-발목 협응을 유지하세요.",
    #     "sources": [ # 답변의 근거가 되는 자료 목록
    #         {"title": "CrossFit Level 1 Training Guide", "url": "https://example.com/cf-level1.pdf"},
    #         {"title": "CrossFit Journal - Clean Mechanics", "url": "https://journal.crossfit.com/example-clean"}
    #     ]
    # }
    # # 챗봇 응답을 대화 기록에 추가
    # app["chat_history"].append({"role": "assistant", "content": resp["answer_text"], "sources": resp["sources"]})
    # # 챗봇 응답에 포함된 근거 자료들을 전역 라이브러리에 추가
    # add_sources(resp["sources"])

    # # UI 반환: 갱신된 채팅 히스토리, 상태 메시지, 링크 패널, 근거 허브 탭 내용
    # updated_sources_md = sources_md()

    # 실제 문서 QA - VectorDB
    email = app["user_session"]["user_id"]
    rag_result = qa_chain.invoke({"query": user_msg})
    answer = rag_result["result"]

    # Q&A 로그 테이블에도 저장
    log_qa(email, user_msg, answer)

    app.setdefault("chat_history", [])
    app["chat_history"].append([user_msg, safe_str(answer)])
    # app["chat_history"].append({"role": "user", "content": user_msg})
    # app["chat_history"].append({"role": "assistant", "content": safe_str(answer)})

    updated_sources_md = sources_md()
    return app["chat_history"], "", gr.update(value=updated_sources_md), gr.update(value=updated_sources_md)

def download_history() -> gr.update:
    """
    현재까지의 챗봇 대화 기록을 JSON 파일로 저장하고,
    이를 다운로드 컴포넌트에 바인딩하여 사용자가 다운로드할 수 있도록 합니다.

    Returns:
        gr.update: Gradio File 컴포넌트의 업데이트 객체 (파일 경로 및 가시성).
    """
    try:
        data = export_chat_json() # 대화 기록을 JSON 문자열로 가져옴
        fname = tmp_filename()    # 고유한 임시 파일명 생성
        # 생성된 파일명으로 파일을 쓰고, 인코딩을 UTF-8로 지정하여 한글 깨짐 방지
        with open(fname, "w", encoding="utf-8") as f:
            f.write(data)
        # Gradio File 컴포넌트의 value를 파일 경로로, visible을 True로 업데이트
        return gr.update(value=fname, visible=True)
    except Exception as e:
        # 파일 저장 중 오류 발생 시 에러 로깅 및 파일 컴포넌트 숨김
        print(f"Error downloading history: {e}")
        return gr.update(value=None, visible=False)

# ------------------------------------------------------------------------------
# 영상 코칭 (데모) 기능
# ------------------------------------------------------------------------------
def analyze_video(pose_type: str, video: str) -> tuple:
    """
    사용자가 업로드한 운동 영상을 분석하여 코칭 피드백을 제공합니다.
    (현재는 모킹된 결과를 사용하며, 실제 환경에서는 영상 분석 백엔드를 호출합니다.)

    Args:
        pose_type (str): 분석할 자세 유형 (예: "파워 스내치").
        video (str): 사용자가 업로드한 비디오 파일의 경로.

    Returns:
        tuple: (user_video_url, ref_video_url, metrics_text, coaching_text, video_history)
               Gradio UI 컴포넌트 업데이트에 필요한 값들.
    """
    # 1. 로그인 상태 확인
    if not app["user_session"]["auth"]:
        return None, None, "로그인 후 이용해 주세요.", "", ""
    # 2. 자세 선택 여부 확인
    if not pose_type:
        return None, None, "자세를 선택해 주세요.", "", ""
    # 3. 영상 업로드 여부 확인
    if video is None:
        return None, None, "영상(mp4)을 업로드해 주세요.", "", ""

    # --------------------------------------------------------------------------
    # 실제 환경에서는 이 부분에서 영상 분석 백엔드(예: 컴퓨터 비전 모델)를 호출하여
    # 영상에서 자세를 분석하고 메트릭 및 코칭 피드백을 받아오게 됩니다.
    # 현재는 데모를 위해 고정된 모킹(mocking) 결과를 사용합니다.
    # --------------------------------------------------------------------------
    result = {
        "ref_video_url": "https://cdn.example.com/ref_snatch.mp4", # 참조 영상 URL (데모용)
        "user_overlay_video_url": video,  # 사용자가 업로드한 영상의 Gradio 내부 경로 또는 URL
        "metrics": { # 분석된 지표
            "score": 88,
            "bar_path_deviation": 3.0,
            "tempo": 1.05,
            "labels": ["주의"] # 주의사항 등 추가 라벨
        },
        "coaching_text": "바벨이 몸에서 멀어집니다. 가슴을 열고 바를 몸 가까이 끌어올리세요." # 코칭 메시지
    }
    # 영상 분석 기록을 전역 video_history에 추가
    app["video_history"].append({
        "ts": now_iso(),       # 분석 시각
        "pose": pose_type,     # 분석된 자세 유형
        "metrics": result["metrics"],   # 분석 지표
        "coaching": result["coaching_text"] # 코칭 메시지
    })

    # 지표를 표시할 텍스트 형식으로 가공
    metrics_text = (
        f"점수: {result['metrics']['score']} | "
        f"궤적편차: {result['metrics']['bar_path_deviation']} | "
        f"템포: {result['metrics']['tempo']}"
    )
    # UI 반환: 사용자 영상, 참조 영상, 지표 텍스트, 코칭 텍스트, 영상 히스토리
    return result["user_overlay_video_url"], result["ref_video_url"], metrics_text, result["coaching_text"], app["video_history"]

# ------------------------------------------------------------------------------
# 개인 맞춤 추천 기능
# ------------------------------------------------------------------------------
def gen_recommend(level: str, goal: str, freq: int, gear: list[str]) -> tuple:
    """
    사용자의 운동 수준, 목표, 빈도, 장비 정보를 바탕으로 맞춤형 운동 계획을 생성합니다.
    생성된 계획은 JSON 형태로 반환되고 챗봇 히스토리에도 추가됩니다.

    Args:
        level (str): 사용자 운동 수준 (예: "초보", "중급").
        goal (str): 사용자 운동 목표 (예: "체지방 감량", "근력 향상").
        freq (int): 주당 운동 횟수.
        gear (list[str]): 사용 가능한 장비 목록.

    Returns:
        tuple: (plan_json_string, status_message)
               JSON 형식의 운동 계획 문자열과 챗봇 전송 성공 메시지.
    """
    # 1. 로그인 상태 확인
    if not app["user_session"]["auth"]:
        return "로그인 후 이용해 주세요.", ""

    # 사용자 입력 정보를 요약
    summary = (
        f"{level} 수준, 목표: {goal}, 주 {int(freq)}회, "
        f"장비: {', '.join(gear) if gear else '없음'}"
    )
    # --------------------------------------------------------------------------
    # 실제 환경에서는 이 부분에서 사용자 입력과 과거 데이터를 바탕으로
    # AI 모델이 개인화된 운동 계획을 생성하게 됩니다.
    # 현재는 데모를 위해 고정된 예시 계획을 사용합니다.
    # --------------------------------------------------------------------------
    plan = {
        "summary": summary, # 요약 정보
        "wod": [ # Workout of the Day (오늘의 운동) 계획
            "WOD A: 파워클린 테크닉 + AMRAP 10분(스윙/버피/싱글언더)",
            "WOD B: 스쿼트클린 EMOM 10분 + 코어 서킷",
            "WOD C: 파워스내치 포지션 드릴 + 컨디셔닝 인터벌"
        ],
        "stretch": ["둔근/햄스트링 스트레칭 5분", "흉추 모빌리티 3분", "발목 가동성 3분"], # 추천 스트레칭
        "notes": ["통증 발생 시 중단", "무게보다 기술 우선", "세트 간 충분한 휴식"] # 주의사항
    }
    # 생성된 추천 기록을 전역 recommend_history에 추가
    app["recommend_history"].append({
        "ts": now_iso(),                # 추천 생성 시각
        "inputs": [level, goal, int(freq), gear], # 추천 생성에 사용된 입력값
        "plan": plan                    # 생성된 운동 계획
    })

    # 챗봇 히스토리에 전송할 메시지 생성
    to_chat = (
        f"[개인추천]\n{plan['summary']}\n- " +
        "\n- ".join(plan["wod"]) +
        "\n스트레칭: " + ", ".join(plan["stretch"])
    )
    app["chat_history"].append({"role": "assistant", "content": to_chat}) # 챗봇 히스토리에 추가

    # UI 반환: JSON 형식의 운동 계획 문자열, 챗봇 전송 성공 메시지
    return json.dumps(plan, ensure_ascii=False, indent=2), "챗봇 히스토리에 전송되었습니다."

# ------------------------------------------------------------------------------
# 용어/규칙 검색 기능
# ------------------------------------------------------------------------------
def search_glossary(q: str, cat: str) -> str:
    """
    크로스핏 용어집에서 사용자의 검색어와 카테고리에 맞는 용어를 검색합니다.

    Args:
        q (str): 검색할 키워드.
        cat (str): 검색할 카테고리 (예: "용어", "프로그램", "전체").

    Returns:
        str: 검색 결과 목록을 Markdown 형식으로 반환하거나, 결과가 없을 경우 메시지 반환.
    """
    # 검색어를 소문자로 변환하고 앞뒤 공백 제거
    q_norm = (q or "").strip().lower()
    # 카테고리 유효성 검사 및 기본값 설정
    cat_str = str(cat or "전체")
    if cat_str not in {"전체", "용어", "프로그램"}:
        cat_str = "전체" # 유효하지 않은 카테고리면 '전체'로 간주

    rows = [] # 검색 결과를 저장할 리스트
    for it in app["glossary"]: # 용어집의 각 항목 순회
        # 1. 검색어 조건 확인: 검색어가 비어있거나, 용어 또는 설명에 검색어가 포함되어 있는지 확인
        ok_q = (not q_norm) or (q_norm in it["term"].lower()) or (q_norm in it["desc"].lower())
        # 2. 카테고리 조건 확인: '전체'이거나 항목의 카테고리가 일치하는지 확인
        ok_c = (cat_str == "전체") or (it["category"] == cat_str)

        # 두 조건 모두 만족하면 결과에 추가
        if ok_q and ok_c:
            rows.append(f"- {it['term']} ({it['category']}): {it['desc']}")

    # 결과가 있으면 Markdown 리스트로 반환, 없으면 메시지 반환
    return "\n".join(rows) if rows else "결과가 없습니다."

# ------------------------------------------------------------------------------
# 단위 변환기 / 식단/회복 가이드 / 인증 정보 / 멘토링
# ------------------------------------------------------------------------------
def convert_weight(value: str, unit: str) -> str:
    """
    킬로그램(kg)과 파운드(lb) 간의 무게 단위를 변환합니다.

    Args:
        value (str): 변환할 무게 값 (숫자 문자열).
        unit (str): 변환 방향 (예: "kg→lb", "lb→kg").

    Returns:
        str: 변환된 결과 문자열 또는 유효성 검사 실패 메시지.
    """
    try:
        v = float(value) # 입력 값을 부동소수점으로 변환
    except ValueError:
        return "숫자를 입력해 주세요." # 숫자가 아닌 경우 에러 메시지

    if unit == "kg→lb":
        return f"{v:.2f} kg = {v * 2.2046226218:.2f} lb"
    elif unit == "lb→kg":
        return f"{v:.2f} lb = {v / 2.2046226218:.2f} kg"
    else:
        return "올바른 단위 변환 방향을 선택해 주세요."

def diet_recovery(weight_band: str, pref: str, allergy: str) -> str:
    """
    사용자 정보에 기반한 간단한 식단 및 회복 가이드라인을 제공합니다.

    Args:
        weight_band (str): 사용자 체중 대역 (예: "<60kg", "60~80kg").
        pref (str): 식성 선호도 (예: "고단백", "채식").
        allergy (str): 알레르기 정보 (옵션).

    Returns:
        str: 식단 및 회복 가이드라인 문자열 또는 로그인 필요 메시지.
    """
    # 1. 로그인 상태 확인
    if not app["user_session"]["auth"]:
        return "로그인 후 이용해 주세요."

    # --------------------------------------------------------------------------
    # 실제 환경에서는 사용자 데이터와 영양학적 지식을 바탕으로
    # 개인화된 식단 추천이 이루어질 수 있습니다.
    # 현재는 데모를 위해 고정된 일반적인 가이드라인을 제공합니다.
    # --------------------------------------------------------------------------
    return "\n".join([
        f"- 체중대: {weight_band}, 선호: {pref}, 알레르기: {allergy or '없음'}",
        "- 운동 후: 유청 단백질 + 바나나 + 물 500ml 섭취",
        "- 식단 예시: 단백질 중심 식사(닭가슴살/두부), 복합탄수화물(현미/고구마), 채소 다량 섭취",
        "- 회복: 하루 7~9시간 충분한 수면, 가벼운 스트레칭으로 혈액 순환 및 근육 회복 촉진"
    ])

def cert_info() -> str:
    """
    크로스핏 관련 온램프 및 인증/챌린지 정보를 제공합니다.

    Returns:
        str: 인증 및 챌린지 정보 요약 문자열.
    """
    return "\n".join([
        "- 온램프 프로그램: 2~4주 과정으로 크로스핏의 기초 동작, 용어, 안전 수칙 교육을 제공하여 초보자가 적응하도록 돕습니다.",
        "- 레벨 테스트: 기본적인 기술 숙련도와 기초 체력을 평가하여 현재 실력 수준을 파악합니다.",
        "- 자격 인증: 교육 수강 후 평가를 통과하면 자격을 얻게 되며, 정기적인 갱신이 필요합니다."
    ])

def mentoring_preset(topic: str) -> str:
    """
    선택된 주제에 대한 멘토링 메시지를 제공하고 이를 챗봇 히스토리에 추가합니다.

    Args:
        topic (str): 멘토링 주제 (예: "첫 수업 긴장", "페이스 조절").

    Returns:
        str: 멘토링 메시지가 챗봇에 전송되었음을 알리는 메시지.
    """
    presets = { # 주제별 미리 정의된 멘토링 메시지
        "첫 수업 긴장": "호흡을 길게 가져가세요. 첫 세트는 60% 강도로 시작하고, 코치와 아이컨택으로 신호를 맞춰보세요.",
        "페이스 조절": "초반엔 70% 정도로 출발하여 체력을 아끼세요. 호흡 리듬은 4초 흡기 - 4초 호기를 유지하고, 타이머를 활용해 체크포인트를 설정하는 것이 좋습니다.",
        "목표 설정": "4주 주기로 작은 소목표를 설정하세요. 동작의 기술 습득을 우선하고, 주당 3회 이상 꾸준히 운동하여 일관성을 유지하는 것이 중요합니다."
    }
    # 선택된 주제에 맞는 메시지를 가져오거나, 없으면 기본 메시지 반환
    msg = presets.get(topic, "필요한 주제를 선택해 주세요.")
    # 챗봇 히스토리에 멘토링 메시지 추가
    app["chat_history"].append({"role": "assistant", "content": f"[멘토링] {msg}"})
    return f"챗봇에 전송됨: {msg}"

### Admin tab 함수

In [11]:
# ==== 상태 테이블 추출 ====
def sqlite_file_info(p):
    if not os.path.isfile(p):
        return "-", "-", "-"
    size = os.path.getsize(p) // 1024
    date = datetime.fromtimestamp(os.path.getmtime(p)).strftime("%Y-%m-%d %H:%M:%S")
    return os.path.basename(p), f"{size} KB", date

def get_all_db_rows():
    curr_p = os.path.join(chroma_dir, "chroma.sqlite3")
    curr_row = ["0", "현재 DB", *sqlite_file_info(curr_p), "백업", ""]
    backup_rows = []
    for idx, ver in enumerate(sorted(os.listdir(chroma_backup_dir), reverse=True), 1):
        sqlite_path = os.path.join(chroma_backup_dir, ver, "chroma.sqlite3")
        fn, sz, dt = sqlite_file_info(sqlite_path)
        backup_rows.append([str(idx), ver, fn, sz, dt, "롤백", "삭제"])
    return [curr_row] + backup_rows

def backup_db(desc):
    now = datetime.now().strftime("%Y%m%d-%H%M%S")
    ver_folder = f"{now}-{desc.strip()}" if desc.strip() else now
    tgt = os.path.join(chroma_backup_dir, ver_folder)
    shutil.copytree(chroma_dir, tgt)
    return get_all_db_rows(), f"📦 백업 완료: {ver_folder}"

def handle_rollback(row_idx):
    all_rows = get_all_db_rows()
    if row_idx == 0:
        return get_all_db_rows(), "⚠️ 현재 DB는 롤백 대상 아님"
    ver = all_rows[int(row_idx)][1]
    src = os.path.join(chroma_backup_dir, ver)
    if not os.path.exists(src):
        return get_all_db_rows(), "❌ 백업본이 존재하지 않습니다!"
    if os.path.exists(chroma_dir):
        shutil.rmtree(chroma_dir)
    shutil.copytree(src, chroma_dir)
    return get_all_db_rows(), f"✅ 복구 완료: {ver}"

def handle_delete(row_idx):
    all_rows = get_all_db_rows()
    if row_idx == 0:
        return get_all_db_rows(), "⚠️ 현재 DB는 삭제 불가"
    ver = all_rows[int(row_idx)][1]
    path = os.path.join(chroma_backup_dir, ver)
    if not os.path.exists(path):
        return get_all_db_rows(), "❌ 이미 삭제된 백업본입니다!"
    shutil.rmtree(path)
    return get_all_db_rows(), f"🗑️ 백업본 삭제 완료: {ver}"

def do_action(idx, desc, act_type):
    rows = get_all_db_rows()
    if idx < 0 or idx >= len(rows):
        return rows, "❌ 올바른 행 번호를 선택하세요."
    if act_type == "백업":
        new_table, msg = backup_db(desc)
        return new_table, msg
    elif act_type == "롤백":
        new_table, msg = handle_rollback(int(idx))
        return new_table, msg
    elif act_type == "삭제":
        new_table, msg = handle_delete(int(idx))
        return new_table, msg
    else:
        return rows, "❓ 지원하지 않는 작업"

In [12]:
# ------------------------------------------------------------------------------
# Gradio UI 빌더
# ------------------------------------------------------------------------------
def build_demo() -> gr.Blocks:
    """
    Gradio를 사용하여 크로스핏 코칭 데모 웹 애플리케이션의 UI를 구축합니다.
    다양한 기능 탭과 인터랙티브 컴포넌트들을 포함합니다.

    Returns:
        gr.Blocks: 구축된 Gradio 블록 앱 객체.
    """
    # Gradio Blocks 컨테이너 시작: 앱 전체의 제목 설정
    with gr.Blocks(title="CrossFit 코칭 데모 (UI 조정판)") as demo:
        session = gr.State({})

        # ==================== 진입 페이지 (로그인/회원가입) ====================
        # 초기에는 이 페이지가 보이고, 로그인 성공 시 숨겨집니다.
        with gr.Column(visible=True) as entry_page:
            gr.Markdown("### 로그인 / 회원가입")
            with gr.Row(): # 입력 필드들을 가로로 배치
                email = gr.Textbox(label="이메일", placeholder="you@example.com", scale=2, type="email")
                name = gr.Textbox(label="닉네임", placeholder="닉네임(선택)", scale=1)
                pwd = gr.Textbox(label="비밀번호", type="password", placeholder="비밀번호", scale=2)
            with gr.Row(): # 버튼들을 가로로 배치
                login_btn = gr.Button("로그인", variant="primary", scale=2) # 주요 버튼 강조
                signup_btn = gr.Button("회원가입", scale=1)
                # demo_btn = gr.Button("데모 계정으로 입장", scale=2)
            entry_status = gr.Markdown("") # 로그인/회원가입 상태 메시지 표시 영역

        # ====================== 메인 페이지 (기능 탭) ======================
        # 로그인 성공 시 이 페이지가 보이고, 진입 페이지가 숨겨집니다.
        with gr.Column(visible=False) as main_page:
            # Welcome 메시지와 로그아웃 버튼을 Main_page의 첫 자식으로 직접 배치하여 수직 스택
            welcome = gr.Markdown("", elem_id="welcome_message_area")
            logout_btn = gr.Button("로그아웃", size="sm", elem_id="logout_button")

            # 이제 왼쪽 메뉴와 오른쪽 탭 콘텐츠를 하나의 Row로 묶습니다.
            with gr.Row(elem_id="main_content_row"):
                # ----------- 좌측 메뉴 영역 -----------
                with gr.Column(scale=1, elem_id="left_menu_column"):
                    gr.Markdown("#### 메뉴")
                    # Markdown을 사용하여 메뉴 항목 나열
                    gr.Markdown("""
- 챗봇(Q&A)
- 영상 코칭
- 개인 맞춤 추천
- 용어/규칙
- 식단/회복
- 인증 안내
- 멘토링
- 근거 자료 허브
                    """)

                    gr.Markdown("#### KG↔LB 계산기")
                    w_val = gr.Textbox(label="무게 값", placeholder="예: 60")
                    w_dir = gr.Radio(choices=["kg→lb", "lb→kg"], value="kg→lb", label="변환 방향")
                    w_btn = gr.Button("변환")
                    w_out = gr.Markdown()

                    gr.Markdown("#### 근거 자료 허브(요약)")
                    evidence_brief = gr.Markdown(value="", elem_id="evidence_brief_summary")

                # ----------- 우측 탭 콘텐츠 영역 -----------
                with gr.Column(scale=3, elem_id="right_tabs_column"):
                    with gr.Tabs(elem_id="main_tabs"):
                        # ----------- 탭 1: 챗봇(Q&A) -----------
                        with gr.Tab("챗봇(Q&A)", elem_id="chatbot_tab"):
                            with gr.Row():
                                user_input = gr.Textbox(label="질문", placeholder="예) 스쿼트 클린 무릎 각도는?", scale=4)
                                send_btn = gr.Button("전송", variant="primary", scale=1)
                            # 챗봇 대화 내용 표시, height를 고정하여 UI 안정화
                            chat_view = gr.Chatbot(label="답변 및 채팅 히스토리", height=420, render_markdown=True)
                            links_panel = gr.Markdown("링크 모아보기", label="연관 자료 링크", elem_id="links_panel_chatbot") # 챗봇 응답과 관련된 링크 표시
                            with gr.Row():
                                download_btn = gr.Button("히스토리 내려받기", elem_id="download_button")
                                # 다운로드될 파일의 경로를 표시하며, 초기에는 보이지 않음
                                download_file = gr.File(label="다운로드 파일", visible=False, elem_id="download_file_output")

                        # ----------- 탭 2: 영상 코칭 -----------
                        with gr.Tab("영상 코칭", elem_id="video_coaching_tab"):
                            pose = gr.Dropdown(
                                ["파워 스내치", "스쿼트 클린", "파워 클린"], # 분석 가능한 자세 목록
                                label="분석할 자세 선택",
                                info="분석하고자 하는 운동 자세를 선택해주세요.",
                                interactive=True # 사용자 선택 가능
                            )
                            vfile = gr.Video(label="사용자 영상 업로드(mp4)", sources=["upload"]) # 영상 업로드 컴포넌트
                            analyze_btn = gr.Button("영상 분석 실행", variant="primary")
                            with gr.Row():
                                user_v = gr.Video(label="사용자 영상 분석 결과") # 분석된 사용자 영상 (오버레이 등)
                                ref_v = gr.Video(label="참조(레퍼런스) 영상") # 올바른 자세를 보여주는 참조 영상
                            metrics = gr.Markdown("지표", label="분석 지표", elem_id="metrics_output") # 분석 결과 지표 표시
                            coaching = gr.Textbox("코칭", label="맞춤 코칭 피드백", lines=5, interactive=False) # 코칭 메시지 표시
                            history = gr.JSON(label="영상 분석 히스토리(현재 세션)", elem_id="video_history_json_output") # JSON 형식으로 영상 분석 기록 표시

                        # ----------- 탭 3: 개인 맞춤 추천 -----------
                        with gr.Tab("개인 맞춤 추천", elem_id="recommendation_tab"):
                            level = gr.Radio(
                                ["초보", "중급", "상급"],
                                value="초보", # 초기 선택값
                                label="경험 수준",
                                info="현재 운동 경험 수준을 선택해주세요."
                            )
                            goal = gr.Radio(
                                ["체지방 감량", "근력 향상", "기술 습득"],
                                value="기술 습득",
                                label="주요 운동 목표",
                                info="가장 중요한 운동 목표를 선택해주세요."
                            )
                            freq = gr.Slider(
                                1, 6, step=1, value=3, # 최소 1, 최대 6, 1단계씩 증가, 초기값 3
                                label="주당 운동 횟수",
                                info="일주일에 몇 번 운동하고 싶으신가요?"
                            )
                            gear = gr.CheckboxGroup(
                                ["덤벨", "케틀벨", "바벨", "로워링 밴드"],
                                label="사용 가능한 장비 선택",
                                info="현재 활용 가능한 운동 장비를 모두 선택해주세요."
                            )
                            rec_btn = gr.Button("맞춤 추천 생성", variant="primary")
                            rec_out = gr.Code(label="추천 결과(JSON)", language="json", interactive=False) # 생성된 추천 계획을 JSON 코드로 표시
                            rec_status = gr.Markdown("", elem_id="recommendation_status") # 추천 생성 상태 메시지 표시 (예: "챗봇에 전송되었습니다.")

                        # ----------- 탭 4: 용어/규칙/운동법 -----------
                        with gr.Tab("용어/규칙/운동법", elem_id="glossary_tab"):
                            q = gr.Textbox(label="검색어", placeholder="예) WOD, 스내치")
                            cat = gr.Dropdown(
                                ["전체", "용어", "프로그램"],
                                value="전체",
                                label="검색 카테고리",
                                info="검색할 용어의 카테고리를 선택해주세요."
                            )
                            search_btn = gr.Button("용어 검색")
                            glo_out = gr.Markdown("용어집 검색 결과", elem_id="glossary_output") # 검색 결과 표시

                        # ----------- 탭 5: 식단/회복 -----------
                        with gr.Tab("식단/회복", elem_id="diet_recovery_tab"):
                            weight_band = gr.Dropdown(
                                ["<60kg", "60~80kg", ">80kg"],
                                value="60~80kg",
                                label="현재 체중대",
                                info="현재 체중이 속하는 구간을 선택해주세요."
                            )
                            pref = gr.Dropdown(
                                ["선호 없음", "고단백", "채식"],
                                value="고단백",
                                label="식성 선호도",
                                info="평소 식단 선호도를 선택해주세요."
                            )
                            allergy = gr.Textbox(
                                label="알레르기 정보 (선택 사항)",
                                placeholder="예) 유제품, 견과류"
                            )
                            diet_btn = gr.Button("회복 가이드 생성", variant="primary")
                            diet_out = gr.Markdown("식단 및 회복 가이드", elem_id="diet_output") # 생성된 가이드 표시

                        # ----------- 탭 6: 인증/챌린지 안내 -----------
                        with gr.Tab("인증/챌린지 안내", elem_id="certification_tab"):
                            cert_btn = gr.Button("정보 요약 보기")
                            cert_out = gr.Markdown("크로스핏 인증 및 챌린지 정보", elem_id="certification_output") # 정보 표시

                        # ----------- 탭 7: 멘토링(초보 심리/동기) -----------
                        with gr.Tab("멘토링(초보 심리/동기)", elem_id="mentoring_tab"):
                            topic = gr.Radio(
                                ["첫 수업 긴장", "페이스 조절", "목표 설정"],
                                value="첫 수업 긴장",
                                label="멘토링 주제 선택",
                                info="도움이 필요한 주제를 선택해주세요."
                            )
                            mt_btn = gr.Button("멘토링 메시지 챗봇에 전송", variant="secondary") # 챗봇으로 메시지 전송 버튼
                            mt_out = gr.Markdown("", elem_id="mentoring_status_output") # 전송 결과 메시지 (예: "챗봇에 전송됨: ...")

                        # ----------- 탭 8: 근거 자료 허브 -----------
                        with gr.Tab("근거 자료 허브", elem_id="evidence_hub_tab"):
                            # 현재까지 적립된 모든 근거 자료 링크를 Markdown 형태로 표시
                            evidence_full = gr.Markdown("수집된 근거 자료", elem_id="evidence_full_output")

                        #  ----------- 탭 9: 관리자 VectorDB 관리 -----------
                        with gr.Tab("관리자 VectorDB 관리", visible=False) as admin_tab:
                            gr.Markdown("## 🗄️ **VectorDB 관리자 대시보드**")
                            backup_desc = gr.Textbox(label="백업 설명(선택)", placeholder="ex) major-update, 실험 등")
                            db_table = gr.Dataframe(
                                value=get_all_db_rows(),
                                headers=["행 번호", "구분(버전명)", "파일명", "크기", "최종수정", "롤백", "삭제"],
                                interactive=False,
                                label="DB 관리",
                                row_count=(len(get_all_db_rows()), "fixed"),
                                col_count=(7, "fixed"),
                            )
                            op_result = gr.Textbox(label="실행 결과", lines=3, interactive=False)

                            # 행 선택
                            row_select = gr.Number(
                                label="행 번호 선택 (0=현재, 1~=백업행)",
                                minimum=0, value=0, precision=0
                            )

                            # 버튼 - 자동분기
                            action_type = gr.Radio(choices=["백업", "롤백", "삭제"], value="백업", label="실행 작업 선택")
                            action_btn = gr.Button("작업 실행")

                            def table_reset():
                                return get_all_db_rows(), ""

                            demo.load(table_reset, None, [db_table, op_result])

                            action_btn.click(
                                do_action,
                                [row_select, backup_desc, action_type],
                                [db_table, op_result]
                            )


        # ====================== 이벤트 바인딩 (UI와 기능 함수 연결) ======================

        # -------------------- 진입 페이지 이벤트 --------------------
        # 로그인 버튼 클릭 시 do_login 함수 실행
        # 입력: email, name, pwd / 출력: entry_page(visible), main_page(visible), welcome(value)
        login_btn.click(do_login, [email, name, pwd], [entry_page, main_page, welcome, admin_tab])
        # 회원가입 버튼 클릭 시 do_signup 함수 실행 (데모에서는 do_login과 동일)
        signup_btn.click(do_signup, [email, name, pwd], [entry_page, main_page, welcome])

        # 데모 계정 입장 버튼 클릭 시 고정된 데모 계정 정보로 do_login 함수 실행
        # demo_btn.click(lambda: do_login("demo@demo", "데모", "x"), None, [entry_page, main_page, welcome])

        # 로그아웃 버튼 클릭 시 do_logout 함수 실행
        # 출력: entry_page(visible), main_page(visible), entry_status(value)
        logout_btn.click(do_logout, None, [entry_page, main_page, entry_status, email, name, pwd])

        # -------------------- 기능 탭별 이벤트 --------------------
        # 챗봇: 전송 버튼 클릭 시 send_chat 함수 실행
        # 입력: user_input / 출력: chat_view(value), entry_status(value), links_panel(value), evidence_full(value)
        send_btn.click(send_chat, [user_input], [chat_view, entry_status, links_panel, evidence_full])
        # 히스토리 다운로드 버튼 클릭 시 download_history 함수 실행
        # 출력: download_file(value, visible)
        download_btn.click(download_history, None, [download_file])

        # 영상 코칭: 분석 실행 버튼 클릭 시 analyze_video 함수 실행
        # 입력: pose, vfile / 출력: user_v(value), ref_v(value), metrics(value), coaching(value), history(value)
        analyze_btn.click(analyze_video, [pose, vfile], [user_v, ref_v, metrics, coaching, history])

        # 개인 맞춤 추천: 추천 생성 버튼 클릭 시 gen_recommend 함수 실행
        # 입력: level, goal, freq, gear / 출력: rec_out(value), rec_status(value)
        rec_btn.click(gen_recommend, [level, goal, freq, gear], [rec_out, rec_status])

        # 용어/규칙 검색: 검색 버튼 클릭 시 search_glossary 함수 실행
        # 입력: q, cat / 출력: glo_out(value)
        search_btn.click(search_glossary, [q, cat], [glo_out])

        # 식단/회복: 가이드 생성 버튼 클릭 시 diet_recovery 함수 실행
        # 입력: weight_band, pref, allergy / 출력: diet_out(value)
        diet_btn.click(diet_recovery, [weight_band, pref, allergy], [diet_out])

        # 인증/챌린지 안내: 요약 보기 버튼 클릭 시 cert_info 함수 실행
        # 입력: None / 출력: cert_out(value)
        cert_btn.click(lambda: cert_info(), None, [cert_out]) # 람다 함수로 래핑하여 인자 없이 호출

        # 멘토링: 멘토링 메시지 전송 버튼 클릭 시 mentoring_preset 함수 실행
        # 입력: topic / 출력: mt_out(value)
        mt_btn.click(mentoring_preset, [topic], [mt_out])

        # 무게 변환기 이벤트 바인딩 추가
        w_btn.click(convert_weight, [w_val, w_dir], [w_out])

        # 초기 로드 시 근거 허브 탭 세팅 및 요약본 갱신
        def refresh_evidence_on_load():
            """Gradio UI가 처음 로드될 때 근거 자료 허브 내용 및 요약본을 갱신합니다."""
            full_md = sources_md()
            return full_md, full_md # evidence_full과 evidence_brief를 동시에 업데이트
        demo.load(refresh_evidence_on_load, None, [evidence_full, evidence_brief]) # demo.load는 앱 시작 시 1회 실행

    return demo # 구축된 Gradio 앱 반환

# ------------------------------------------------------------------------------
# Gradio 앱 빌드 및 실행 (Colab 환경 최적화)
# ------------------------------------------------------------------------------

# 이전에 실행 중이던 Gradio 인스턴스가 있다면 종료하여 포트 충돌을 방지합니다.
try:
    demo.close()
except NameError:
    # 'demo'가 아직 정의되지 않았을 수 있으므로 NameError 처리
    pass
except Exception as e:
    print(f"Error closing previous demo instance: {e}")

demo = build_demo() # 위에서 정의한 함수를 호출하여 Gradio 앱 UI를 구축

# Colab에서 사용 가능한 포트를 탐색합니다. 기본 포트 7860이 사용 중일 경우 다른 포트를 찾습니다.
port = find_free_port() or 7861 # 사용 가능한 포트가 없으면 7861을 기본값으로 사용

# Gradio 앱을 실행합니다.
demo.launch(
    server_name="0.0.0.0",        # 모든 네트워크 인터페이스에서 접근 가능하도록 설정
    server_port=port,             # 탐색된 포트 또는 기본 포트 사용
    debug=True,                   # 디버그 모드 활성화 (에러 메시지 상세 출력)
    show_error=True,              # 사용자에게 에러 메시지를 표시
    share=True,                   # Colab 환경에서 외부에서 접근 가능한 Public URL 생성
    prevent_thread_lock=True      # Gradio 스레드가 메인 스레드를 잠그는 것을 방지
)

# Public URL이 Colab 출력창에 표시될 것임을 안내합니다.
print(f"Gradio 앱이 포트 {port}에서 실행 중입니다. Public 링크를 확인해주세요.")

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e12e22dc60478744c2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7920 <> https://e12e22dc60478744c2.gradio.live
Gradio 앱이 포트 7920에서 실행 중입니다. Public 링크를 확인해주세요.
